# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import SimpleITK as sitk

import pandas as pd
import matplotlib.pyplot as plt

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)

/home/eddyod/programming/pipeline/src


In [ ]:
#from library.controller.sql_controller import SqlController
#from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, load_transformation
#from library.atlas.brain_structure_manager import BrainStructureManager
#from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
#    read_image, write_image


In [45]:
def get_origins(animal):
    """
    Fetches the origins from disk. The data is already scaled to 10um.
    """

    origins = {}
    dirpath = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}/origin'
    if not os.path.exists(dirpath):
        return origins
    files = sorted(os.listdir(dirpath))
    for file in files:
        structure = Path(file).stem
        filepath = os.path.join(dirpath, file)
        origin = np.loadtxt(filepath)
        origins[structure] = origin
    return origins

def compute_affine_transformation(src_pts, dst_pts):
    """
    src_pts: Nx3 array of points in brain A
    dst_pts: Nx3 array of points in brain B
    
    Returns:
        T  : 4x4 affine matrix mapping src → dst
        err: RMS error after transform
    """

    src_pts = np.asarray(src_pts)
    dst_pts = np.asarray(dst_pts)
    assert src_pts.shape == dst_pts.shape
    assert src_pts.shape[0] >= 4  # minimum points for 3D affine

    N = src_pts.shape[0]

    # Build homogeneous design matrix: [x y z 1]
    X = np.hstack([src_pts, np.ones((N, 1))])        # Nx4
    Y = dst_pts                                       # Nx3

    # Solve X * A = Y  → A is 4×3 matrix
    A, *_ = np.linalg.lstsq(X, Y, rcond=None)

    # Convert to 4×4 transform
    T = np.eye(4)
    T[:3, :4] = A.T

    # Apply transform to src
    src_h = np.hstack([src_pts, np.ones((N, 1))])    # Nx4
    dst_pred = (T @ src_h.T).T[:, :3]

    # Compute RMS error
    rmse = np.sqrt(np.mean(np.sum((dst_pts - dst_pred)**2, axis=1)))

    return T, rmse, dst_pred

def make_homogeneous(mat3):
    M = np.eye(4, dtype=float)
    M[:3, :3] = mat3
    return M

def diag4(spacing):
    s = np.array(spacing, dtype=float)
    D = np.eye(4, dtype=float)
    D[0,0], D[1,1], D[2,2] = s[0], s[1], s[2]
    return D

def sitk_affine_to_matrix4x4(tx: sitk.Transform):
    params = list(tx.GetParameters())
    
    # First 9 values → affine matrix
    A = np.array(params[:9]).reshape(3,3)

    # Last 3 values → translation
    t = np.array(params[9:12]).reshape(3)

    # Build homogeneous 4×4
    M = np.eye(4)
    M[:3,:3] = A
    M[:3, 3] = t

    return M

In [ ]:
moving_name = 'MD594'
fixed_name = 'AtlasV8'
moving_all = get_origins(moving_name)
fixed_all = get_origins(fixed_name)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('10N_L','10N_R')
good_keys = set(common_keys) - set(bad_keys)
moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(good_keys))
transformation_matrix, rmse, predictions = compute_affine_transformation(moving_src, fixed_src)
data_path = "/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration"
transformation_matrix_path = os.path.join(data_path, moving_name, f'{moving_name}_{fixed_name}_10.0um')
np.save(transformation_matrix_path, transformation_matrix)
print(transformation_matrix)
print('rmse', rmse)

df_list = []
predicted_dict = {}
for structure in good_keys:    
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    predicted0 = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(predicted0, fixed0)]   
    row = [structure, np.round(moving0,2), np.round(fixed0,2), np.round(predicted0,2), np.round(difference,2)]
    df_list.append(row)
    predicted_dict[structure] = predicted0

In [ ]:
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS: 237.06805950085737 observations: 37
# MD585 to MD594 152.06606097021333 observations: 51
# MD585 to Allen 263.31435 observations: 37
# MD589 to Allen 260.02 observations: 37
# MD594 to Allen 250.79 observations: 37

In [ ]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#comtfm = np.array([824.6051918494063, 80.83004570523167, 363.4390121956811])
transformed_structure = affine_transform_point(com, transformation_matrix)

print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure,2)}')
diff = transformed_structure - fixed_all[structure]
#comdiff = comtfm - fixed_all[structure]

print(f'{moving_name}->{fixed_name} error {structure} {diff}')
#print(f'{moving_name}->{fixed_name} tfm error {structure} {comdiff}')

In [ ]:
columns = ['structure', moving_name, fixed_name, 'predicted', 'difference']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(2)
df.sort_values(by=['structure'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)
#20	3N_R	[1079.0, 531.0, 485.0]	[910.0, 380.0, 17.0]	[873.0, 358.0, 298.0]	[-37.0, -21.0, 281.0]	284.3851
#7	4N_L	[1135.0, 529.0, 423.0]	[959.0, 378.0, 544.0]	[923.0, 362.0, 356.0]	[-36.0, -16.0, -188.0]	192.0327

In [ ]:
registered_origin_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/registered_origin'
for structure, origin in moving_all.items():
    new_origin = affine_transform_point(origin, transformation_matrix)
    new_origin_path = os.path.join(registered_origin_path, f'{structure}.txt')
    print(structure, np.round(origin,2), np.round(new_origin,2))
    np.savetxt(new_origin_path, new_origin)

In [ ]:
v_m = moving image voxel coordinate (i,j,k) (your fiducials)
s_m = (2,2,20) µm = moving image spacing
s_affine = (10,10,10) µm = the spacing that the affine was computed with / expects
A = 4×4 affine transform you have (homogeneous). You must know what A maps to/from (see cases below).
s_f = fixed image spacing (if you want final voxel indices)

In [ ]:
A = np.array(
[[ 9.53477969e-01,  1.09410189e-01,  2.63478745e-02,  4.55867422e+02],
 [-7.88806971e-02,  9.05905687e-01, -5.44966440e-02,  3.52298187e+02],
 [-1.20042733e-02,  5.02446347e-02,  9.32364556e-01, -6.78249849e+01],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]
)
print(A.shape, A.dtype)

In [ ]:
fiducials_vox = np.array([[36913.835658476906, 30879.23503839053, 280.]], dtype=float)

A = np.array(
[[ 9.53477969e-01,  1.09410189e-01,  2.63478745e-02,  4.55867422e+02],
 [-7.88806971e-02,  9.05905687e-01, -5.44966440e-02,  3.52298187e+02],
 [-1.20042733e-02,  5.02446347e-02,  9.32364556e-01, -6.78249849e+01],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]
)

In [88]:
um = 25.0
allen_size = (728,400,456)
fixed = sitk.Image(allen_size, sitk.sitkFloat32)
fixed.SetOrigin((0,0,0))
fixed.SetSpacing((um, um, um))
print('spacing', fixed.GetSpacing())
print('origin', fixed.GetOrigin())
print('dir', fixed.GetDirection())


spacing (25.0, 25.0, 25.0)
origin (0.0, 0.0, 0.0)
dir (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)


In [98]:
regpath = "/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration"
moving_name = "DK55"
#transform_path = os.path.join(regpath, moving_name, f"{moving_name}_Allen_{um}x{um}x{um}um.tfm")
transform_path = os.path.join(regpath, moving_name, "DK55_to_Allen_affine.tfm")
print(transform_path)
sitk_transform = sitk.ReadTransform(transform_path)

/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/DK55/DK55_to_Allen_affine.tfm


In [99]:
sitk_transform.GetParameters()

(0.7899430196433067,
 -0.25892724294511893,
 0.026358084908220763,
 -0.029557349057782244,
 0.82513762960629,
 -0.037050847530486396,
 -0.010258155194592392,
 0.04869510622071581,
 0.94758146793511,
 3162.502608957253,
 3037.5022746932304,
 -850.0003472537259)

In [100]:
A = sitk_affine_to_matrix4x4(sitk_transform)
np.round(A,2)

array([[ 7.9000e-01, -2.6000e-01,  3.0000e-02,  3.1625e+03],
       [-3.0000e-02,  8.3000e-01, -4.0000e-02,  3.0375e+03],
       [-1.0000e-02,  5.0000e-02,  9.5000e-01, -8.5000e+02],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

In [93]:
# goal is 271, 300, 97
#This works!
# user values
moving_index = (28037/32,25992/32,122)
moving_spacing = (10.4, 10.4, 20)   # µm

# put them into microns
phys_moving = (moving_index[0]*moving_spacing[0],
               moving_index[1]*moving_spacing[1],
               moving_index[2]*moving_spacing[2])
print("moving physical (µm) =", phys_moving)
phys_fixed = sitk_transform.TransformPoint(phys_moving)
print("fixed physical (µm) =", np.round(phys_fixed))
fixed_index = fixed.TransformPhysicalPointToIndex(phys_fixed)
print(f'moved to fixed space {um}um {fixed_index}')


moving physical (µm) = (9112.025, 8447.4, 2440)
fixed physical (µm) = [13868. 11626.  2005.]
moved to fixed space 25.0um (555, 465, 80)


In [73]:
phys_moving

(9125.0, 6375.0, 2350.0)

In [92]:
# fiducial points from the moving brain
for phys_m in [phys_moving]:
    phys_f = sitk_transform.TransformPoint(phys_m)
    idx_f = fixed.TransformPhysicalPointToIndex(phys_f)
    print("phys fixed", phys_f)
    print("fixed_index:", idx_f)


phys fixed (13867.93356178531, 11626.417937878321, 2004.9556320272095)
fixed_index: (555, 465, 80)


In [49]:
fiducials_vox = np.array([(28037, 25992,122)])
print(fiducials_vox.shape)
s_m = (0.325, 0.325, 20.0)    # moving spacing in microns (given)
s_affine = (25.0, 25.0, 25.0)  # spacing that affine expects / was computed with
# If you know the fixed spacing, set it; otherwise set to None
s_f = (25.0, 25.0, 25.0)  # example fixed spacing (change if different) or None

# Suppose A is the 4x4 affine you have (replace with your matrix)
# Here we create an example identity-like A for demonstration:
#A = np.eye(4)  # replace with your real 4x4 affine matrix
A = sitk_affine_to_matrix4x4(sitk_transform)

# Build scale matrices (homogeneous)
S_m = diag4(s_m)
S_aff = diag4(s_affine)
S_f = diag4(s_f) if s_f is not None else None

(1, 3)


In [64]:
fiducial_indices

array([[28037, 25992,   122]])

In [66]:
moving_spacing = (0.325, 0.325, 20.0)   # microns
moving_origin = (0.0, 0.0, 0.0)     # set from your image if available
moving_direction = (1.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,1.0)  # 3x3 row-major; set from image if available

# Load the transform saved from registration (example .tfm)
transform = sitk_transform

# helper to convert index -> physical (microns)
def index_to_physical(index, spacing, origin=(0,0,0), direction=(1,0,0, 0,1,0, 0,0,1)):
    idx = np.asarray(index, dtype=float)
    sp = np.asarray(spacing, dtype=float)
    # apply spacing
    scaled = idx * sp  # in microns from image origin-aligned axes
    # apply direction 3x3
    dir_mat = np.array(direction).reshape(3,3)
    phys_rel = dir_mat.dot(scaled)
    origin = np.asarray(origin, dtype=float)
    return tuple(origin + phys_rel)

# convert and transform
physical_points = [index_to_physical(pt, moving_spacing, moving_origin, moving_direction)
                   for pt in fiducial_indices]

# apply transform (SimpleITK expects points in same physical units as images; here microns)
transformed_physical = [transform.GetInverse().TransformPoint(p) for p in physical_points]
print(transformed_physical)

[(4405.474587253084, 4555.097711818389, 3088.2144534508207)]


In [65]:
physical_points

[(9112.025, 8447.4, 2440.0)]

In [ ]:
allenpath = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/ALLEN771602/ALLEN771602_25.0x25.0x25.0um_sagittal.tif'
allen = sitk.ReadImage(allenpath)
print(allen.GetOrigin())
print(allen.GetSize())
allen.SetSpacing((25.0, 25.0, 25.0))
outpath = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration/Allen/ALLEN771602_25.0x25.0x25.0um_sagittal.nii'
sitk.WriteImage(allen, outpath)